# MNIST数字認識 (畳み込みNNの利用)

## 必要なパッケージをインポート

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt

## グローバル定数の設定

In [ ]:
batch_size = 200                # ミニバッチサイズ
sgd_lr = 0.1 # SGDの学習率

## データローダの準備 (MNISTデータのダウンロードも含む)

In [ ]:
root = '.' # mnistデータの置き場所
download = True
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST(root=root, train=True, transform=trans, download=download)
test_set = datasets.MNIST(root=root, train=False, transform=trans)
# ローダの準備
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

## データの中身を表示してみる

In [ ]:
data_iter = iter(train_loader)

In [ ]:
images, labels = data_iter.next()
npimg = images[0].numpy()
npimg = npimg.reshape((28, 28))
plt.imshow(npimg, cmap='gray')
print('Label:', labels[0].item())

## モデルの定義


In [ ]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(1, 32, 3) # 28x28x1 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3) # 26x26x32 -> 24x24x64 
        self.pool = nn.MaxPool2d(2, 2) # 24x24x64 -> 12x12x64
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x)) #　第１畳み込み層
        x = F.relu(self.conv2(x)) # 第２畳み込み層
        x = self.pool(x) # プーリング層
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x)) #全結合層 1
        x = self.fc2(x) # 全結合層 2
        return F.log_softmax(x, dim=1)

## 訓練ループ

学習には結構時間がかかるので、本を読むなりコーヒーを飲むなりして気長に待ちましょう。。。

In [ ]:
model = Net() # モデルのインスタンス生成
optimizer = optim.SGD(model.parameters(), lr=sgd_lr)
running_loss = 0.0
i = 0
for loop in range(3): # 3エポックの訓練
    for (input, target) in train_loader:
        i = i + 1
        optimizer.zero_grad()    # optimizerの初期化
        output = model(input)     # 推論計算
        loss = F.nll_loss(output, target) # 損失関数の定義
        loss.backward()             # バックプロパゲーション(後ろ向き計算)
        optimizer.step()            # パラメータ更新
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%5d] loss: %.3f' %
                  (i + 1, running_loss / 100))
            running_loss = 0.0

## 精度の評価

In [ ]:
correct =  0 # 正解数
count = 0 # 試行数
with torch.no_grad():
    for (input, target) in test_loader:
        output = model(input)     # 推論計算
        pred = output.argmax(dim=1)
        correct += pred.eq(target.data).sum()
        count += batch_size
print ('accuracy = ', float(correct)/float(count)) # 正解率の表示

## テスト認識を実施してみる

In [ ]:
test_iter = iter(test_loader)

In [ ]:
input, labels = test_iter.next()
output = model(input)     # 推論計算
pred = output.argmax(dim=1)
plt.imshow(input[0].numpy().reshape(28, 28), cmap='gray')
print('True      Label:', labels[0].item())
print('Estimated Label:', pred[0].item())

---------------------


## 演習問題 3-5 (チャレンジ問題)

上記のコードを実行しつつ、理解せよ。この問題についてはレポートによる報告は必要ない。

## 演習問題 3-6 (チャレンジ問題)

正答率99%超を目指して工夫してみよ。達成できた場合、その工夫と実行結果をレポートにまとめよ。 

-------------------
